In [1]:
import pandas as pd
import numpy as np

In [17]:
#read in the results and artist table csv
artists = pd.read_csv('artists_table.csv')
results = pd.read_csv('artist_scrape_results.csv')

In [18]:
#remove junk from ends and drop holding columns
results[['junk1', 'biography_hold']] = results['bio'].str.split("'",1, expand=True)
results[['Biography', 'junk2']] = results['biography_hold'].str.rsplit("'",1, expand=True)
results = results.drop(columns=['junk1', 'biography_hold', 'junk2', 'bio', 'header', 'Unnamed: 0'])

In [19]:
#delete all values that contain the phrase:Other reasons this message may be displayed:\n
results['Biography'] = np.where(results['Biography'].str.contains('Other reasons this'), 'Biography could not be found', results['Biography'])
results['Biography'] = np.where(results['Biography'].str.contains('may refer to:'), 'Biography could not be found', results['Biography'])

In [20]:
#remove any images that are not .jpg
results['image'] = np.where(results['image'].str.contains('.svg'), 'image could not be found', results['image'])

In [21]:
#merge image and bio into artists table, drop any unneeded columns, fill in any na
artists = artists.merge(results, left_on='Full_name', right_on='name')
artists = artists.drop(columns=['Unnamed: 0', 'name'])
artists['Biography'] = artists['Biography'].fillna('Biography could not be found')
artists['image'] = artists['image'].fillna('Image could not be found')

In [22]:
artists = artists.drop_duplicates()

In [25]:
artists = artists.fillna('Unknown')

In [27]:
#read final artist table into csv 
artists.to_csv('artists_table_final.csv', index=False)